In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import scipy.optimize
import scipy.ndimage
from vega import VegaInterface
from astropy.io import ascii

In [ ]:
#vega = VegaInterface('vega_cfg/old-grid-repro.ini')
vega = VegaInterface('/global/homes/b/bzh/clamato-xcorr/vega_cfg/main_example.ini')
# assert np.all(vega.data['qsoxlya'].mask)

In [ ]:
vega.data['qsoxlya'].mask.shape

In [ ]:
vega.params['bias_QSO'] = 5
vega.params['beta_QSO'] = vega.params['growth_rate'] / vega.params['bias_QSO']
vega.params['drp_QSO'] = 0
vega.params['par_sigma_smooth'] = 0

In [ ]:
vega.priors

In [ ]:
print(vega.fiducial['z_eff'])

In [ ]:
# For example:
rp_vals = np.arange(0.1, 40, 0.2)
# print(rp_vals[:10], rp_vals[-10:])
rt_vals = np.arange(0.1, 30, 0.2)
# print(rt_vals[:10], rt_vals[-10:])

xcorr_shape = (len(rt_vals), len(rp_vals))
print(xcorr_shape)

# Why is this reversed in the example 'simple model' notebook?
rt_grid, rp_grid = np.meshgrid(rt_vals, rp_vals)

# Overwrite rp_rt grid:
vega.corr_items['qsoxlya'].rp_rt_grid = [rp_grid.flatten(), rt_grid.flatten()]
vega.corr_items['qsoxlya'].z_grid = 2.3
vega.corr_items['qsoxlya'].has_metals = False

In [ ]:
print(vega.corr_items['qsoxlya'].rp_rt_grid)

In [ ]:
%time
vega.fiducial['save-components'] = True
model = vega.compute_model()

In [ ]:
model_xcorr = vega.models['qsoxlya'].xi_distorted['smooth']['core'].reshape(*reversed(xcorr_shape))
#model_xcorr += vega.params['bao_amp'] * vega.models['qsoxlya'].xi_distorted['peak']['core'].reshape(*xcorr_shape)

In [ ]:
model_xcorr.shape

In [ ]:
plt.pcolormesh(rt_vals, rp_vals, model_xcorr, cmap='jet_r', vmin=-0.04, vmax=0)
plt.xlabel('Transverse [Mpc/h]')
plt.ylabel('Parallel [Mpc/h]')
plt.colorbar()

In [ ]:
model_sum_reflected = np.sum(model_xcorr, axis=1)
model_sum_reflected = np.concatenate([model_sum_reflected[::-1], model_sum_reflected])
parallel_reflected = np.concatenate([-rp_vals[::-1], rp_vals])
plt.plot(parallel_reflected, model_sum_reflected * -1)
# print(np.std(model_sum_reflected) * 0.2)

In [ ]:
def gauss(x, *p):
    A, mu, sigma, C = p
    # sigma = 40
    dist = np.minimum(np.abs(x - (mu + sigma)), np.abs(x - (mu - sigma)))
    dist[x > mu + sigma] = 0
    dist[x < mu - sigma] = 0
    #return A * dist**3 + C
    return A * scipy.ndimage.gaussian_filter(dist**3, sigma=vega.params['par_sigma_smooth'] / 0.2) + C
    #return A*np.exp(-(x-mu)**2/(2.*sigma**2)) + C

fit_coeff, _ = scipy.optimize.curve_fit(gauss, parallel_reflected, model_sum_reflected, p0=[(np.min(model_sum_reflected) + 0.25) / 40**3, 0, 40, -0.25], maxfev=5000)
print(fit_coeff)
plt.plot(parallel_reflected, model_sum_reflected, color='red', label='Vega model')
plt.plot(parallel_reflected, gauss(parallel_reflected, *fit_coeff), color='black', label='Fit function')
plt.xlabel(r'$\pi$ [Mpc/h]')
plt.ylabel('Summed cross-corr')
plt.title(f'bias = {vega.params["bias_QSO"]}, redshift disp = {vega.params["par_sigma_smooth"] / 2}')
plt.legend()

# Old model grid

In [ ]:
grid_model_flat = ascii.read('/global/homes/b/bzh/clamato-xcorr/data/model/model_grid_v1/linear_cross_b0.8_s0.501187.txt', 
                             format='no_header', names=['rt', 'rp', 'xi_lin', 'xi_conv'])
grid_model = np.zeros_like(model_xcorr)
grid_model[:] = grid_model_flat['xi_conv'].reshape(*model_xcorr.shape, order='F')

In [ ]:
plt.pcolormesh(rt_vals, rp_vals, grid_model, cmap='jet_r', vmin=-0.04, vmax=0)
plt.colorbar()

# Comparison & Residuals

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
kwargs = {'cmap': 'jet_r', 'vmin': -0.15, 'vmax': 0}
axes[0].pcolormesh(rt_vals, rp_vals, model_xcorr, **kwargs)
axes[1].pcolormesh(rt_vals, rp_vals, grid_model, **kwargs)
#fig.colorbar(None, cmap=kwargs['cmap'], norm=matplotlib.colors.Normalize(vmin=kwargs['vmin'], vmax=kwargs['vmax']))

In [ ]:
plt.pcolormesh(rt_vals, rp_vals, np.abs((model_xcorr - grid_model) /  grid_model) * 100, vmax=200, cmap='plasma')
plt.colorbar(label='% difference')

In [ ]:
plt.pcolormesh(rt_vals, rp_vals, np.abs((model_xcorr - grid_model) /  grid_model) * 100, vmax=100, cmap='plasma')
plt.colorbar(label='% difference')
plt.xlim(0, 5)
plt.ylim(0, 5)